In [31]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1


In [37]:
def match_faces(target_image, gallery_folder, num_matches=5):
    mtcnn = MTCNN(keep_all=True)
    resnet = InceptionResnetV1(pretrained='vggface2').eval()

    def get_embeddings(image):
        img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        face_tensors = mtcnn(img)
        if face_tensors is None:
            return None
        embeddings = resnet(face_tensors)
        return embeddings.detach().numpy()

    # Load the target image and extract embeddings
    target_embeddings = get_embeddings(target_image)

    if target_embeddings is None:
        print("No faces found in the target image.")
        return []

    # Load the gallery images and extract embeddings
    gallery_embeddings = []
    gallery_images = []
    for filename in tqdm(os.listdir(gallery_folder), desc='Processing gallery images'):
        image = cv2.imread(os.path.join(gallery_folder, filename))
        if image is not None:
            gallery_images.append(image)
            gallery_embeddings.append(get_embeddings(image))

    if len(gallery_embeddings) == 0:
        print("No faces found in the gallery images.")
        return []

    # Compute the similarity between target and gallery embeddings
    similarities = []
    for i, gallery_embedding in enumerate(gallery_embeddings):
        similarity = np.dot(target_embeddings, gallery_embedding.T)
        similarities.append((i, similarity[0][0]))

    # Sort the results by similarity
    sorted_results = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_matches = sorted_results[:num_matches]

    if len(top_matches) == 0:
        print("No matches found.")
    else:
        for i, (index, similarity) in enumerate(top_matches):
            matched_image = gallery_images[index]
            matched_filename = os.listdir(gallery_folder)[index]
            print(f"Match {i+1}: Filename {matched_filename}, Similarity {similarity}")

    return top_matches

In [41]:
target_image = cv2.imread('C:\\Users\\Stephen Otieno\\Downloads\\Owako PHOTO.jpg')
gallery_folder = 'images'
results = match_faces(target_image, gallery_folder)

Processing gallery images: 100%|█████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.58it/s]

Match 1: Filename Boniface james.jpg, Similarity 0.5515308976173401
Match 2: Filename Emmanuel Kirui.jpg, Similarity 0.5278165340423584
Match 3: Filename John Kamau.jpg, Similarity 0.4929405450820923
Match 4: Filename Sheryl Achieng.jpg, Similarity 0.4246913194656372
Match 5: Filename Daniel james.jpg, Similarity 0.42221271991729736
